In [ ]:
import pickle

try:
    f = open('./var_pickle', 'rb')
    fronts = pickle.load(f)
    f.close()
  
except EOFError:
    print("aaaaa")

In [ ]:
import pickle

try:
    f = open('./status_pickle', 'rb')
    car_status = pickle.load(f)
    f.close()
  
except EOFError:
    print("aaaaa")

In [ ]:
car_status

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import os

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.imshow(fronts[0,5,:,:,:].detach().cpu().permute(1, 2, 0).int())

plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(1, 6)
for i in range(6):
    axarr[i].imshow(fronts[0,i,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')

plt.show()

In [ ]:
from models.DemoV2 import DemoNet
from utils.config import GlobalConfig
from utils.datasets.carla_data import CARLA_Data
from torch.utils.data import DataLoader

In [ ]:
config = GlobalConfig()
model = DemoNet(config, 'cuda')
# model = nn.DataParallel(model, device_ids=[0, 1]).cuda()

In [ ]:
model.load_state_dict(torch.load(os.path.join('/root/End2End/log/Demov2/demo', 'best_model.pth')))
model.train = False

In [ ]:
model = model.module

In [ ]:
image = fronts
b, s, c, w, h = image.shape

image = image.view(-1, c, w, h)
for i in range(b * s):

    image[i,0,:,:] = (image[i,0,:,:] - 0.485) / 0.229
    image[i,1,:,:] = (image[i,1,:,:] - 0.456) / 0.224
    image[i,2,:,:] = (image[i,2,:,:] - 0.406) / 0.225

image = image.view(-1, c, w, h)
img_features_total = model.image_encode_net(image)   

_, c, w, h = img_features_total.shape
img_features_total = img_features_total.view(b, s, c, w, h)
img_features = img_features_total[:,:model.post_len,:,:,:]
img_label = img_features_total[:,model.post_len:,:,:,:]
b, s, c, w, h = img_features.shape

# car_status = self.car_status_net(status)
img_features = img_features.view(b, -1, w, h)
post_w = model.post_weight_net(img_features)
post_w = F.normalize(post_w, p=2, dim=1).unsqueeze(2)

img_features = img_features.view(b, s, -1)
post_feature_weight = img_features * post_w

post_feature_weight = post_feature_weight.view(b, s * c, w, h)
pred_features = model.pred_features_net(post_feature_weight) ## (pred_len, w/16 * h/16)

# pred_features = pred_features.view(b, s, c, w, h)
pred_features = pred_features.view(b, s * c, w, h)
pred_w = model.pred_weight_net(pred_features)
pred_w = F.normalize(pred_w, p=2, dim=1).unsqueeze(2)

pred_features = pred_features.view(b, s, -1)
pred_features_weight = pred_features * pred_w

post_feature_weight = post_feature_weight.view(b, s, c, w, h)
pred_features_weight = pred_features_weight.view(b, s, c, w, h)
total_features_weight = torch.cat((post_feature_weight, pred_features_weight), dim=1).view(b, -1)

encode_status = model.status_encode_net(car_status)

pred_ctr = model.pred_ctr_net(total_features_weight, encode_status)
pred_features = pred_features.view(b, s, c, w, h)
# return pred_ctr, pred_features, img_label

In [ ]:
pred_ctr

In [ ]:
sum_img_features_total = img_features_total.squeeze(0).sum(dim=1)

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0)
f, axarr = plt.subplots(1,6)
for i in range(6):
    pic1 = pred_features_weight.sum(dim=2)[0, i, :, :]
    pic1 = pic1.squeeze(0).detach().cpu()
    # plt.imshow(pic1[:, :], cmap='Greys_r')
    axarr[i].imshow(pic1, cmap='Greys_r')

plt.show()